In [18]:
import pandas as pd
import itertools
import numpy as np

In [19]:
df = pd.read_csv("step2_out.csv")
front = df[df.front & (df["average log2 error"] < 10)].copy()
front

,layers,branching factor,average error,average error %,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front
78,"radix26,linear",524288,149.365164,0.009180,7.174567,18361,0.009180,281018376,276824072,nmcecb82faae80a9205fdb,True,False,True
99,"radix22,linear",524288,1202.095267,0.109135,9.298148,218271,0.109135,29360136,25165832,nmfa710a8dc67ac3313a8f,True,False,True
104,"radix22,linear",2097152,1208.882305,0.109135,9.135692,218271,0.109135,67108872,50331656,nm0e0edc4d1772833e44ec,True,False,True
106,"radix26,linear",1048576,129.910627,0.009180,6.695010,18361,0.009180,293601288,285212680,nm1051edb8eb3c4850071e,True,False,True
112,"radix22,linear",262144,1421.024570,0.169592,9.473658,339184,0.169592,23068680,20971528,nm22c753529c7921d89536,True,False,True
120,"radix26,linear",131072,274.164242,0.029432,8.520465,58863,0.029432,271581192,270532616,nm425718530d6822ba063d,True,False,True
127,"radix26,linear",2097152,121.809124,0.010065,6.360232,20129,0.010065,318767112,301989896,nm5224d02427c12afc4e35,True,False,True
134,"radix22,linear",1048576,1214.666648,0.109135,9.196831,218271,0.109135,41943048,33554440,nm62b7c64259882d940086,True,False,True
143,"radix26,linear",262144,188.899311,0.029432,7.783807,58863,0.029432,274726920,272629768,nm7755b9180e88b20aaa91,True,False,True
155,"radix22,linear",131072,1430.878489,0.169592,9.757601,339184,0.169592,19922952,18874376,nm8d1751e28dacc6051714,True,False,True


In [15]:
front = front.to_dict("record")

In [17]:
retained = []
for rmi1 in front:
    for rmi2 in front:
        if rmi1["layers"] == rmi2["layers"] and rmi1["branching factor"] == rmi2["branching factor"]:
            continue
        
        if abs(rmi1["average log2 error"] - rmi2["average log2 error"]) < 0.01:
            if rmi1["size binary search"] > rmi2["size binary search"]:
                break
                
        if abs(rmi1["size binary search"] - rmi2["size binary search"]) < 1000:
            if rmi1["average log2 error"] > rmi2["average log2 error"]:
                break
    else:
        retained.append(rmi1)
        
retained = pd.DataFrame(retained)
retained["diff"] = [1000] + list(np.diff(retained["average log2 error"].to_numpy()))
selected = retained.sort_values("diff", ascending=False)[0:10]
selected = selected.sort_values("size binary search", ascending=False)
selected

,layers,branching factor,average error,average error %,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
6,"radix26,linear",2097152,121.809124,0.010065,6.360232,20129,0.010065,318767112,301989896,nm5224d02427c12afc4e35,True,False,True,-2.160233
3,"radix26,linear",1048576,129.910627,0.009180,6.695010,18361,0.009180,293601288,285212680,nm1051edb8eb3c4850071e,True,False,True,-2.440682
0,"radix26,linear",524288,149.365164,0.009180,7.174567,18361,0.009180,281018376,276824072,nmcecb82faae80a9205fdb,True,False,True,1000.000000
8,"radix26,linear",262144,188.899311,0.029432,7.783807,58863,0.029432,274726920,272629768,nm7755b9180e88b20aaa91,True,False,True,-1.413024
5,"radix26,linear",131072,274.164242,0.029432,8.520465,58863,0.029432,271581192,270532616,nm425718530d6822ba063d,True,False,True,-0.953194
2,"radix22,linear",2097152,1208.882305,0.109135,9.135692,218271,0.109135,67108872,50331656,nm0e0edc4d1772833e44ec,True,False,True,-0.162456
7,"radix22,linear",1048576,1214.666648,0.109135,9.196831,218271,0.109135,41943048,33554440,nm62b7c64259882d940086,True,False,True,2.836599
1,"radix22,linear",524288,1202.095267,0.109135,9.298148,218271,0.109135,29360136,25165832,nmfa710a8dc67ac3313a8f,True,False,True,2.123580
4,"radix22,linear",262144,1421.024570,0.169592,9.473658,339184,0.169592,23068680,20971528,nm22c753529c7921d89536,True,False,True,2.778648
9,"radix22,linear",131072,1430.878489,0.169592,9.757601,339184,0.169592,19922952,18874376,nm8d1751e28dacc6051714,True,False,True,1.973794


In [4]:
DATASET = "wiki_ts_200M_uint64"
tmp = selected.reset_index().drop(columns="index").reset_index()
tmp["namespace"] = tmp.apply(lambda x: DATASET + "_" + str(x["index"]), axis=1)
spec = {"configs" : tmp[["layers", "branching factor", "namespace", "binary"]].to_dict("record")}

import json
print(json.dumps(spec))

{"configs": [{"layers": "cubic,linear", "branching factor": 786432, "namespace": "wiki_ts_200M_uint64_0", "binary": true}, {"layers": "cubic,linear", "branching factor": 49152, "namespace": "wiki_ts_200M_uint64_1", "binary": true}, {"layers": "cubic,linear", "branching factor": 24576, "namespace": "wiki_ts_200M_uint64_2", "binary": true}, {"layers": "normal,linear", "branching factor": 12288, "namespace": "wiki_ts_200M_uint64_3", "binary": true}, {"layers": "loglinear,linear", "branching factor": 8192, "namespace": "wiki_ts_200M_uint64_4", "binary": true}, {"layers": "normal,linear", "branching factor": 6144, "namespace": "wiki_ts_200M_uint64_5", "binary": true}, {"layers": "cubic,linear", "branching factor": 3072, "namespace": "wiki_ts_200M_uint64_6", "binary": true}, {"layers": "cubic,linear", "branching factor": 384, "namespace": "wiki_ts_200M_uint64_7", "binary": true}, {"layers": "cubic,linear", "branching factor": 192, "namespace": "wiki_ts_200M_uint64_8", "binary": true}, {"laye